## Neural Network model for restaurant recomndation ##

In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf

# Load user data from CSV
user_df = pd.read_csv('user.csv')

# Load restaurant data from CSV
restaurant_df = pd.read_csv('restaurant.csv')

# Preprocess data
def preprocess_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data

# Define neural network architectures
def create_user_nn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(258, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(embedding_size)  
    ])
    return model

def create_restaurant_nn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(258, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(embedding_size)  
    ])
    return model

# Train neural networks
def train_user_nn(user_data):
    scaled_user_data = preprocess_data(user_data)
    input_shape = scaled_user_data.shape[1:]
    
    model = create_user_nn(input_shape)
    model.compile(optimizer='adam', loss='mse')
    model.fit(scaled_user_data, user_labels, epochs=num_epochs, batch_size=batch_size)
    
    return model

def train_restaurant_nn(restaurant_data):
    scaled_restaurant_data = preprocess_data(restaurant_data)
    input_shape = scaled_restaurant_data.shape[1:]
    
    model = create_restaurant_nn(input_shape)
    model.compile(optimizer='adam', loss='mse')
    model.fit(scaled_restaurant_data, restaurant_labels, epochs=num_epochs, batch_size=batch_size)
    
    return model

# Compute embeddings
def compute_embeddings(model, data):
    scaled_data = preprocess_data(data)
    embeddings = model.predict(scaled_data)
    return embeddings

# Recommendation prediction
def predict_recommendations(user_embedding, restaurant_embeddings):
    scores = np.dot(user_embedding, restaurant_embeddings.T)
    recommendations = np.argsort(scores)[::-1]  
    return recommendations

# Example usage
# Example usage with optimizations
user_data = user_df[['stars_count']].values
restaurant_data = restaurant_df.drop(columns=['business_id', 'name', 'stars']).values

embedding_size = 32  # Adjust the embedding size to match the label size
user_labels = np.random.rand(user_data.shape[0], embedding_size)  # Random labels for demonstration
restaurant_labels = np.random.rand(restaurant_data.shape[0], embedding_size)

num_epochs = 20  # Increase number of epochs
batch_size = 28  # Increase batch size

# Define and compile neural network models
user_nn = create_user_nn(user_data.shape[1:])
user_nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

restaurant_nn = create_restaurant_nn(restaurant_data.shape[1:])
restaurant_nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')


# Train neural network models
user_nn.fit(preprocess_data(user_data), user_labels, epochs=num_epochs, batch_size=batch_size, verbose=1)
restaurant_nn.fit(preprocess_data(restaurant_data), restaurant_labels, epochs=num_epochs, batch_size=batch_size, verbose=1)

# Compute embeddings
user_embeddings = compute_embeddings(user_nn, user_data)
restaurant_embeddings = compute_embeddings(restaurant_nn, restaurant_data)

# Recommendation prediction using dot product

# Recommendation prediction using sum of squared distances with business details
def predict_recommendations_with_details(user_id, user_embedding, restaurant_embeddings, restaurant_df):
    scores = np.dot(user_embedding, restaurant_embeddings.T)
    recommendations_indices = np.argsort(scores)[::-1]
    recommendations = []

    for index in recommendations_indices:
        business_id = restaurant_df.iloc[index]['business_id']
        business_name = restaurant_df.iloc[index]['name']
        recommendations.append({'user_id': user_id, 'business_id': business_id, 'business_name': business_name})
        
    return recommendations

# Example usage
user_id = '-0vutwuE36iYMqM2wHaStQ'  # Example user ID
user_data_for_prediction = user_df[user_df['user_id'] == user_id][['stars_count']].values
user_embedding = compute_embeddings(user_nn, user_data_for_prediction).squeeze()

recommendations_with_details = predict_recommendations_with_details(user_id, user_embedding, restaurant_embeddings, restaurant_df)
print("Top recommended restaurants with details:")
for recommendation in recommendations_with_details[:10]:
    print(recommendation)




c:\Users\Vrinda\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
9700/9700 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.0879
Epoch 2/20
9700/9700 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.0835
Epoch 3/20
9700/9700 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.0835
Epoch 4/20
9700/9700 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - loss: 0.0835
Epoch 5/20
9700/9700 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.0834
Epoch 6/20
4246/9700 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.0835

## CosSine model for restaurant recommendation ##

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler, StandardScaler

user_data = pd.read_csv('user.csv')
restaurant_data = pd.read_csv('restaurant.csv')

# Step 2: Preprocess the data
# Drop unnecessary columns
user_preferences = user_data.drop(columns=['user_id', 'stars_count'])

# Scale the user preferences using Min-Max Scaling
minmax_scaler = MinMaxScaler()
user_preferences_scaled = minmax_scaler.fit_transform(user_preferences)

# Drop non-numeric columns from restaurant data
restaurant_features = restaurant_data.drop(columns=['business_id', 'name', 'stars'])

# Scale the restaurant features using Standard Scaling
standard_scaler = StandardScaler()
restaurant_features_scaled = standard_scaler.fit_transform(restaurant_features)

# Step 3: Calculate cosine similarity between users and restaurants
# Calculate cosine similarity between users and restaurants
user_restaurant_similarity = cosine_similarity(user_preferences_scaled, restaurant_features_scaled)

# Step 4: Generate recommendations for a given user
# Step 4: Generate recommendations for a given user
def recommend_highest_rated_restaurants(user_id, n=10):
    # Find the index of the user in the user_data
    user_index = user_data[user_data['user_id'] == user_id].index[0]
    # Get the similarity scores for the given user
    user_scores = user_restaurant_similarity[user_index]
    # Find the indices of the top n restaurants with the highest similarity scores
    top_indices = user_scores.argsort()[-n:][::-1]
    # Get the corresponding business IDs, names, and their similarity scores
    recommendations = [(restaurant_data.iloc[i]['business_id'], restaurant_data.iloc[i]['name'], user_scores[i]) for i in top_indices]
    return recommendations

# Example usage
user_id = '--Dz7-yZ5vMuDdlHxfBWVw'  # Provide the user_id for whom you want to make recommendations
recommendations = recommend_highest_rated_restaurants(user_id)

print("Top 10 Recommended Businesses with Cosine Similarity Values:")
for i, (business_id, business_name, similarity_score) in enumerate(recommendations, 1):
    print(f"{i}. Business ID: {business_id}, Business Name: {business_name}, Similarity Score: {similarity_score}")
